In [1]:
#get processed data

import pandas as pd

lut = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/lutein.csv?raw=true")

sug = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/sugars.csv?raw=true")

wat=pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/water.csv?raw=true")

#dropping Name column
lut=lut.drop('name',axis=1)
sug=sug.drop('name',axis=1)
wat=wat.drop('name',axis=1)


KeyError: "['serving_size'] not found in axis"

In [3]:
#replace NaN using MICE algorithm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


imp = IterativeImputer(max_iter=10, random_state=2023)

#lutein mcar
#Need to drop lutein mar and true lutein columns
lutmcar=lut.copy()
lutmcar=lutmcar.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)
imp.fit(lutmcar)
lutmcar2=imp.transform(lutmcar)

lutmcarimp = pd.DataFrame(lutmcar2, columns = lutmcar.columns)

#lutein mar
#Need to drop lutein mcar and true lutein columns
lutmar=lut.copy()
lutmar=lutmar.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)
imp.fit(lutmar)
lutmar2=imp.transform(lutmar)

lutmarimp = pd.DataFrame(lutmar2, columns = lutmar.columns)

#sugars mcar
#Need to drop sugar mar and true sugar columns
sugmcar=sug.copy()
sugmcar=sugmcar.drop(['sugars_mar','sugars'],axis=1)
imp.fit(sugmcar)
sugmcar2=imp.transform(sugmcar)
sugmcarimp = pd.DataFrame(sugmcar2, columns = sugmcar.columns)


#sugars mar
#Need to drop sugar mcar and true sugar columns
sugmar=sug.copy()
sugmar=sugmar.drop(['sugars_mcar','sugars'],axis=1)
imp.fit(sugmar)
sugmar2=imp.transform(sugmar)
sugmarimp = pd.DataFrame(sugmar2, columns = sugmar.columns)


#water mcar
#Need to drop water mar and true water columns
watmcar=wat.copy()
watmcar=watmcar.drop(['water_mar','water'],axis=1)
imp.fit(watmcar)
watmcar2=imp.transform(watmcar)
watmcarimp = pd.DataFrame(watmcar2, columns = watmcar.columns)


#water mar
#Need to drop water mcar and true water columns
watmar=wat.copy()
watmar=watmar.drop(['water_mcar','water'],axis=1)
imp.fit(watmar)
watmar2=imp.transform(watmar)
watmarimp = pd.DataFrame(watmar2, columns = watmar.columns)


In [4]:
#adding back the column with true values

lutmcarimp['true_lut'] = lut['lutein_zeaxanthin'].values
lutmarimp['true_lut'] = lut['lutein_zeaxanthin'].values

sugmcarimp['true_sug'] = sug['sugars'].values
sugmarimp['true_sug'] = sug['sugars'].values

watmcarimp['true_wat'] = wat['water'].values
watmarimp['true_wat'] = wat['water'].values

In [5]:
#calculating rmse for each columns and missing type
import math
import numpy as np


meth='MICE'
results=[]

#lutein mcar
dat='lutein'
dt=lutmcarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mcar'])**2
lutmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmcarrmse)
results.append([dat,'mcar',meth,lutmcarrmse])

#lutein mar
dt=lutmarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mar'])**2
lutmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmarrmse)
results.append([dat,'mar',meth,lutmarrmse])

#sugars mcar
dat='sugars'
dt=sugmcarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mcar'])**2
sugmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmcarrmse)
results.append([dat,'mcar',meth,sugmcarrmse])

#sugars mar
dt=sugmarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mar'])**2
sugmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmarrmse)
results.append([dat,'mar',meth,sugmarrmse])

#water mcar
dat='water'
dt=watmcarimp

dt['sqrt']=(dt['true_wat']-dt['water_mcar'])**2
watmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmcarrmse)
results.append([dat,'mcar',meth,watmcarrmse])

#water mcar
dt=watmarimp

dt['sqrt']=(dt['true_wat']-dt['water_mar'])**2
watmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmarrmse)
results.append([dat,'mar',meth,watmarrmse])

print(results)

RMSE: 428.817930744911
RMSE: 464.55477742759507
RMSE: 5.455392326716083
RMSE: 3.667661134704056
RMSE: 0.36204399923720865
RMSE: 0.25807238991238446
[['lutein', 'mcar', 'MICE', 428.817930744911], ['lutein', 'mar', 'MICE', 464.55477742759507], ['sugars', 'mcar', 'MICE', 5.455392326716083], ['sugars', 'mar', 'MICE', 3.667661134704056], ['water', 'mcar', 'MICE', 0.36204399923720865], ['water', 'mar', 'MICE', 0.25807238991238446]]


In [6]:
res = pd.DataFrame(results, columns = ['Column', 'Missing type','Imputation method','RMSE'])
print(res)

   Column Missing type Imputation method        RMSE
0  lutein         mcar              MICE  428.817931
1  lutein          mar              MICE  464.554777
2  sugars         mcar              MICE    5.455392
3  sugars          mar              MICE    3.667661
4   water         mcar              MICE    0.362044
5   water          mar              MICE    0.258072
